In [36]:
import re
import requests
from openai import OpenAI
from tqdm import tqdm
import json

In [ ]:
def get_ds_res(query):
    api_key = ""
    base_url = ""

    client = OpenAI(
        api_key=api_key,
        base_url=base_url
    )

    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": query,
            }
        ],
        model="deepseek-v3",
        temperature=0.5,
        presence_penalty=0,
        frequency_penalty=0,
        top_p=1,
        response_format={
            'type': 'json_object'
        }
    )
    return chat_completion.choices[0].message.content

In [38]:
dataset = json.load(open("../../dataset/public_qa.json", 'r'))
dataset = dataset[:10000]
input_list = []
input_query = ""
for data in dataset:
    if len(input_query + data['input']) < 10000:
        input_query = input_query + data['input'] + '\n'
    else:
        input_list.append(input_query)
        input_query = data['input']
print(len(input_list))

241


In [39]:
results = []

for data in tqdm(input_list):
    query = f"问答内容：{data}"
    query = '''请总结出以下问答内容中涉及病情诊断、康复指导和心理干预三个领域的问题，
定义：
病情诊断类问题：涉及患者对自身疾病状态的疑问，包括但不限于对手术结果、病理报告、复发风险、转移可能性、后续治疗方案（如放疗、化疗、内分泌治疗、靶向治疗）的理解和咨询。通常关注疾病的性质、范围、发展趋势以及预后。
康复指导类问题：涉及患者在术后恢复过程中遇到的实际问题，包括但不限于伤口护理、疼痛管理、饮食营养、运动锻炼、淋巴水肿预防、日常生活调整等方面。通常关注如何改善身体功能、提高生活质量、减少并发症。
心理干预类问题：涉及患者在患病和治疗过程中产生的心理困扰，包括但不限于焦虑、抑郁、恐惧、失眠、情绪波动、人际关系问题、性生活问题等方面。通常关注如何应对负面情绪、减轻心理压力、重建积极心态。
注意：
1.总结出的问题不应该包含患者个人信息
2.问题内容需要足够简洁
3.每个领域的问题不应该超过10个，如果超过10个就只保留出现频率最高的问题。
EXAMPLE JSON OUTPUT:
{
    "病情诊断": \[
        'question1',
        'question2',
        ...
    \],
    "康复指导": \[
        'question1',
        'question2',
        ...
    \],
    "心理干预": \[
        'question1',
        'question2',
        ...
    \]
}'''
    response = get_ds_res(query)
    results.append(response)

100%|██████████| 241/241 [50:04<00:00, 12.47s/it]


In [53]:
results_json = []
for data in results:
    matchs = re.search(r"```json(.*?)```", data, re.DOTALL)
    if matchs:
        results_json.append(json.loads(matchs.group(1).replace("\'", "\"")))
    else:
        results_json.append(json.loads(data.replace("\'", "\"")))

In [56]:
summaried_qa = {"病情诊断":[],"康复指导":[],"心理干预":[]}

for _, key in enumerate(summaried_qa):
    for data in results_json:
        summaried_qa[key].extend(data[key])
with open('../../dataset/summaried_qa.json', 'w') as file:
    json.dump(summaried_qa, file, ensure_ascii=False)

In [26]:
def summary_2(dataset, key):
    input_list = []
    input_query = ""
    for data in dataset[key]:
        if len(input_query + data) < 5000:
            input_query = input_query + data + "\n"
        else:
            input_list.append(input_query)
            input_query = data + "\n"
    print(len(input_list))
    results = []
    for data in tqdm(input_list, desc=key):
        query = f"问答内容：{data}"
        query += f"请将以上问答内容涉及{key}的信息进行总结。注意：总结出的问题不应该包含患者个人信息，问题要足够简洁，问题数量不能超过30个，问题内容必须与{key}相关！"
        query += '''EXAMPLE JSON OUTPUT:
        [
            "question1",
            "question2",
            ...
        ]'''
        response = get_ds_res(query)
        try:
            results.extend(json.loads(response))
        except:
            response = get_ds_res(query)
            matchs = re.search(r"```json(.*?)```", response, re.DOTALL)
            if matchs:
                results.extend(json.loads(matchs.group(1)))
            else:
                print(response)
    return results

In [21]:
summaried_qa = json.load(open("../../dataset/summaried_qa.json", 'r'))

In [27]:
for _, key in enumerate(summaried_qa):
    summaried_qa[key] = summary_2(summaried_qa, key)

25


病情诊断: 100%|██████████| 25/25 [11:54<00:00, 28.58s/it]


17


康复指导: 100%|██████████| 17/17 [07:53<00:00, 27.85s/it]


14


心理干预: 100%|██████████| 14/14 [06:38<00:00, 28.47s/it]


In [33]:
for _, key in enumerate(summaried_qa):
    summaried_qa[key] = list(set(summaried_qa[key]))
    print(len(summaried_qa[key]))
with open('../../dataset/summaried_qa_2.json', 'w') as file:
    json.dump(summaried_qa, file, ensure_ascii=False)

729
484
380
